In [1]:
import numpy as np
import tensorflow  as tf
from tensorflow.contrib.layers import fully_connected
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import librosa
from google.colab import files
from google.colab import drive
import glob
import pickle
from statistics import mean
import keras
from keras.layers.core import Dense, Permute, Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.regularizers import l2
from sklearn.metrics import confusion_matrix


np.random.seed(42)

Using TensorFlow backend.


In [2]:
drive.mount('/content/Drive/',force_remount =  True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/Drive/


In [0]:
dirpath = "/content/Drive/My Drive/DL project/New Files/Mel Data/" ####Change

In [0]:
#Loading training pickle files
sound_lists_pkl_in = open(dirpath+"Catalan_mel_2500.pkl", "rb")
ca_data_mel = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"French_mel_2500.pkl", "rb")
fr_data_mel = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"English_mel_2500.pkl", "rb")
en_data_mel = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"German_mel_2500.pkl", "rb")
de_data_mel = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"Italian_mel_2500.pkl", "rb")
it_data_mel = pickle.load(sound_lists_pkl_in)



In [0]:
def data_creation(dict):
  data=[]
  mel=[]
  for i in range(2500):
    s = dict['mel'][i]
    mel.append(s)
  mel = np.array(mel)
  for i in range(len(mel)):
    ele = np.array([mel[i],dict['labels'][i]])
    data.append(ele)
  tr_data = np.array(data[:2000])
  te_data = np.array(data[2000:])
  return tr_data, te_data

In [7]:
#Checking data for italian
time=[]
for i in range(2500):
  t = it_data_mel['mel'][i].shape
  time.append(t)
print('min:',min(time))
print('max:',max(time))

min: (215, 128)
max: (242, 128)


In [0]:
#Appending italian with zeros
for i in range(len(it_data_mel['mel'])):
  max_len = 242
  zeros = np.zeros((max_len-it_data_mel['mel'][i].shape[0],it_data_mel['mel'][i].shape[1]))
  it_data_mel['mel'][i] = np.concatenate((it_data_mel['mel'][i],zeros), axis=0)
  

In [0]:
ca_data_mel_tr, ca_data_mel_te = data_creation(ca_data_mel)

In [10]:
print(len(ca_data_mel_tr))
print(len(ca_data_mel_te))

2000
500


In [0]:
#Train Data
# ca_data_mel_tr, ca_data_mel_te = data_creation(ca_data_mel)
en_data_mel_tr, en_data_mel_te = data_creation(en_data_mel)
fr_data_mel_tr, fr_data_mel_te = data_creation(fr_data_mel)
it_data_mel_tr, it_data_mel_te = data_creation(it_data_mel)
de_data_mel_tr, de_data_mel_te = data_creation(de_data_mel)

In [12]:
de_data_mel_tr[0]

array([array([[7.80457434e-07, 8.16006239e-08, 4.96844250e-07, ...,
        8.02388426e-09, 3.53154418e-09, 2.50996637e-10],
       [5.23175602e-07, 2.31100796e-07, 5.46005448e-07, ...,
        6.57729968e-09, 3.46898831e-09, 2.85545132e-10],
       [1.04983865e-06, 2.49915937e-06, 1.65298861e-06, ...,
        5.63895780e-09, 2.19717601e-09, 1.69920772e-10],
       ...,
       [2.71399047e-04, 1.93461394e-04, 2.71555819e-04, ...,
        2.36638217e-03, 3.74422654e-04, 1.84601619e-05],
       [3.68046983e-04, 7.37894870e-04, 8.32881879e-04, ...,
        2.21180844e-03, 3.04823619e-04, 1.94661517e-05],
       [7.51647510e-04, 1.16108362e-03, 1.37731047e-03, ...,
        9.82638044e-04, 1.89948630e-04, 2.71922827e-05]]),
       'German'], dtype=object)

In [0]:
final_data_tr = np.concatenate((ca_data_mel_tr, fr_data_mel_tr, en_data_mel_tr, de_data_mel_tr, it_data_mel_tr), axis = 0)

In [0]:
final_data_te = np.concatenate((ca_data_mel_te, fr_data_mel_te, en_data_mel_te, de_data_mel_te, it_data_mel_te), axis = 0)

In [15]:
#QC
print(np.unique(final_data_tr[:,1], return_counts = True))
print(np.unique(final_data_te[:,1], return_counts = True))


(array(['Catalan', 'English', 'French', 'German', 'Italian'], dtype=object), array([2000, 2000, 2000, 2000, 2000]))
(array(['Catalan', 'English', 'French', 'German', 'Italian'], dtype=object), array([500, 500, 500, 500, 500]))


In [0]:
del(ca_data_mel_te)
del(it_data_mel_te)
del(fr_data_mel_te)
del(en_data_mel_te)
del(de_data_mel_te)

In [0]:
del(ca_data_mel_tr)
del(it_data_mel_tr)
del(fr_data_mel_tr)
del(en_data_mel_tr)
del(de_data_mel_tr)

In [0]:
np.random.shuffle(final_data_tr)
np.random.shuffle(final_data_te)

In [19]:
#Labels

labels_tr =[]
for i in range(len(final_data_tr)):

      labels_tr.append(final_data_tr[i][1])  

labels_tr = np.array(labels_tr)
labels_tr

classnames, indices = np.unique(labels_tr, return_inverse=True)
print(indices)
print(classnames)

final_labels_tr = indices

[3 1 0 ... 1 0 3]
['Catalan' 'English' 'French' 'German' 'Italian']


In [20]:
labels_te =[]
for i in range(len(final_data_te)):

      labels_te.append(final_data_te[i][1])  

labels_te = np.array(labels_te)
labels_te

classnames, indices = np.unique(labels_te, return_inverse=True)
print(indices)
print(classnames)

final_labels_te = indices

[3 1 2 ... 2 3 3]
['Catalan' 'English' 'French' 'German' 'Italian']


In [21]:
final_X_tr =[]
for i in range(len(final_data_tr)):

      final_X_tr.append(final_data_tr[i][0])  

final_X_tr = np.array(final_X_tr)
final_X_tr[0].shape


(242, 128)

In [22]:
final_X_te =[]
for i in range(len(final_data_te)):

      final_X_te.append(final_data_te[i][0])  

final_X_te = np.array(final_X_te)
final_X_te[0].shape


(242, 128)

### Keras - MobileNet

In [0]:
from keras.layers.core import Dense, Permute, Reshape, Flatten
from keras.layers import Dense, Permute, Reshape, Input
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.layers import Dropout, TimeDistributed
from keras import layers
from keras.models import Sequential
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.layers import Input
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D
from keras.applications.mobilenet import MobileNet
from keras.models import Model
import keras

In [0]:
# input_shape = (40,972,1)
input_shape2 = (242,128,1)


In [0]:
tx = np.array(final_X_tr[:8000])
cvx = np.array(final_X_tr[8000:])
ty = np.array(final_labels_tr[:8000])
cvy = np.array(final_labels_tr[8000:])

In [0]:
del(final_data_tr)
del(final_data_te)

In [0]:
del(final_labels_tr)
del(final_X_tr)

In [0]:
cvy_cat = to_categorical(cvy)
ty_cat = to_categorical(ty)

In [30]:
#Resnet Model
base_model = MobileNet(weights = None, include_top= False, input_shape=input_shape2, alpha=1.0, dropout = 0.9)

# for layer in base_model.layers:
#   layer.trainable = False
 
x = base_model.output
# x = GlobalAveragePooling2D()(x)

x = Dropout(0.9)(x)

# x = Permute((2, 1, 3))(x)

# bs, x, y, c = model.layers[-1].output_shape

# model.add(Reshape([x,y*c]))

# model.add(LSTM(512, activation="relu",dropout = 0.7, return_sequences=False))

# x = Permute((2, 1, 3))(x)

_x, _y, _c = [int(s) for s in x._shape[1:]]
x = Reshape((_x, _y*_c))(x)

# x = LSTM(256, return_sequences=False)(x)

x = LSTM(512, activation="tanh", dropout = 0.7, return_sequences=False)(x)

# x = Bidirectional(LSTM(512, return_sequences=False, dropout = 0.7), merge_mode="concat")(x)
# x = Flatten()(x)

# x = Dense(1024, activation="relu")(x)

predictions = Dense(5, activation="softmax")(x)

model = Model(inputs = base_model.input, output = predictions)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.adam(lr=0.0005),
             metrics=["accuracy"])

In [0]:
model.fit(tx.reshape(-1,242,128,1), ty_cat,
          batch_size=100,
          epochs=100,
          verbose=1,
          validation_data=(cvx.reshape(-1,242,128,1),cvy_cat)
         )
          

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 45s 6ms/step - loss: 1.6919 - acc: 0.2140 - val_loss: 1.6477 - val_acc: 0.2555
Epoch 2/100
8000/8000 [==============================] - 37s 5ms/step - loss: 1.6615 - acc: 0.2266 - val_loss: 1.7438 - val_acc: 0.2540
Epoch 3/100
8000/8000 [==============================] - 38s 5ms/step - loss: 1.6423 - acc: 0.2325 - val_loss: 1.6060 - val_acc: 0.2830
Epoch 4/100
8000/8000 [==============================] - 37s 5ms/step - loss: 1.6215 - acc: 0.2460 - val_loss: 1.6187 - val_acc: 0.2420
Epoch 5/100
8000/8000 [==============================] - 37s 5ms/step - loss: 1.6046 - acc: 0.2554 - val_loss: 1.6311 - val_acc: 0.2315
Epoch 6/100
8000/8000 [==============================] - 37s 5ms/step - loss: 1.5849 - acc: 0.2771 - val_loss: 1.6408 - val_acc: 0.2215
Epoch 7/100
8000/8000 [==============================] - 38s 5ms/step - loss: 1.5596 - acc: 0.2893 - val_loss: 1.6653 - val_acc: 0.2560


In [0]:
y_pred = model.predict(cvx.reshape(-1,242,128,1))
matrix = confusion_matrix(cvy_cat.argmax(axis=1), y_pred.argmax(axis=1))

In [0]:
matrix

In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 242, 128, 1)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 243, 129, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 121, 64, 32)       288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 121, 64, 32)       128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 121, 64, 32)       0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 121, 64, 32)       288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 121, 64, 32)       128       
__________